In [4]:
import pandas as pd
import json
import requests
import warnings

warnings.simplefilter(action = 'ignore', category = FutureWarning)

In [5]:
def addr_to_lat_lon(addr, api_key):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + api_key}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = pd.DataFrame(result['documents'][0]['address'], index = [0])[['address_name','x','y']]
    return match_first

In [ ]:
dejang = pd.read_csv('건축물대장_전처리.csv', encoding = 'cp949')
dejang['newPlatPlc'] = dejang['newPlatPlc'].str.lstrip()
dejang = dejang.sort_values(by = 'newPlatPlc')
dejang= dejang.drop(['Unnamed: 0'], axis = 1).reset_index()

api_key = 'bc889895d6ca2c417542ab48cffaef0a'

address_series_whole = dejang['platPlc'].drop_duplicates()
tmp_merged = pd.DataFrame()

In [ ]:
for f_num in range(*[0,len(address_series_whole)//10000]):
    n = f_num*10000
    address_series = address_series_whole[n:n+10000]
    for addr in address_series:
        try:
            # 'platPlc'(구주소)를 input -> kakao api에서 'address_name', 'x', 'y'를 얻는다.
            long_lat = addr_to_lat_lon(addr, api_key)
        except:
            # 지정한 json 위치 내에 'address_name', 'x', 'y'를 찾을 수 없다면 0으로 추가 (주소를 찾지 못하는 경우는 4311건 / 전체 데이터 수에 비해 매우 적음 / drop할 것임)
            long_lat = pd.DataFrame({'address_name':[0], 'x':[0],'y':[0]})
        raw_part = dejang[dejang['platPlc'] == addr]
        raw_part['address_name'] = long_lat.iloc[0,0]
        raw_part['x'] = long_lat.iloc[0,1]
        raw_part['y'] = long_lat.iloc[0,2]
        # 구주소, kakao주소명, 위도, 경도로 구성된 df
        tmp_merged = pd.concat([tmp_merged,raw_part])
    tmp_merged.to_csv(f'data/long_lat_data_{f_num}.csv')